<a href="https://colab.research.google.com/github/09eesx/FeelTrack-GenaritveAIFinalExam/blob/main/final_out_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# === Cihaz Ayarı ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Model Yolları ve Etiketler ===
MODEL_PATHS = {
    "bert": "/content/drive/MyDrive/data/bert_model_20250428_0728",
    "roberta": "/content/drive/MyDrive/data/roberta_model_20250428_0554"
}

LABELS = [
    "depresyon", "intihar eğilimi", "anksiyete", "stres", "kişilik bozukluğu", "bipolar",
    "öfke", "hakaret", "tehdit", "nefret söylemi", "üzgünlük", "umutlu", "sevinç", "normal"
]

CRITICAL_CLASSES = ["intihar eğilimi", "tehdit", "nefret söylemi"]
CONFIDENCE_THRESHOLD = 0.6

# === Model ve Tokenizer Yükleyici ===
def load_model_and_tokenizer(path):
    tokenizer = AutoTokenizer.from_pretrained(path)
    model = AutoModelForSequenceClassification.from_pretrained(path).to(DEVICE).eval()
    return tokenizer, model

# Modelleri yükle
bert_tokenizer, bert_model = load_model_and_tokenizer(MODEL_PATHS["bert"])
roberta_tokenizer, roberta_model = load_model_and_tokenizer(MODEL_PATHS["roberta"])
models_tokenizers = [(bert_tokenizer, bert_model), (roberta_tokenizer, roberta_model)]

# === Ensemble Prediction Fonksiyonu ===
def smart_ensemble_predict(text, models_tokenizers):
    total_probs = np.zeros(len(LABELS))

    for tokenizer, model in models_tokenizers:
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(DEVICE)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=1).cpu().numpy()[0]
        total_probs += probs

    avg_probs = total_probs / len(models_tokenizers)
    predicted_idx = int(np.argmax(avg_probs))
    predicted_label = LABELS[predicted_idx]
    confidence = avg_probs[predicted_idx]

    # Kritik sınıf alarmı kontrolü
    alerts = [(LABELS[i], prob) for i, prob in enumerate(avg_probs)
              if LABELS[i] in CRITICAL_CLASSES and prob > 0.5]

    # Güven eşiği kontrolü
    final_decision = predicted_label if confidence >= CONFIDENCE_THRESHOLD else "Emin Değil"

    probs_dict = {LABELS[i]: round(float(prob), 3) for i, prob in enumerate(avg_probs)}

    return {
        "final_decision": final_decision,
        "predicted_label": predicted_label,
        "confidence": round(float(confidence), 4),
        "all_class_probs": probs_dict,
        "alerts": alerts
    }

def hybrid_analysis(text, models_tokenizers):
    results = {
        "bert_result": smart_ensemble_predict(text, [(bert_tokenizer, bert_model)]),
        "roberta_result": smart_ensemble_predict(text, [(roberta_tokenizer, roberta_model)]),
        "ensemble_result": smart_ensemble_predict(text, models_tokenizers)
    }
    return results


In [ ]:
def parse_user_texts(file_path):
    user_texts = []
    current_user_text = []
    inside_user = False

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line_strip = line.strip()
            # User satırı başlangıcı
            if line_strip.startswith("User "):
                # Eğer önceden bir User metni varsa onu listeye ekle
                if current_user_text:
                    user_texts.append(" ".join(current_user_text).strip())
                    current_user_text = []

                inside_user = True
                # Kullanıcının o satırdaki ilk cümlesini de ekle
                # Örnek: "User 1:" kısmını kaldırmak için split
                first_part = line_strip.split(":", 1)
                if len(first_part) > 1:
                    current_user_text.append(first_part[1].strip())

            # Ellie satırı başlangıcı veya boş satır gelirse User metni bitmiş sayılır
            elif line_strip.startswith("Ellie ") or line_strip == "":
                if inside_user and current_user_text:
                    user_texts.append(" ".join(current_user_text).strip())
                    current_user_text = []
                inside_user = False

            # Eğer şu an User metni içerisindeysek devam eden satırları ekle
            elif inside_user:
                current_user_text.append(line_strip)

    # Dosya sonunda hala User metni varsa ekle
    if current_user_text:
        user_texts.append(" ".join(current_user_text).strip())

    return user_texts


# Ana kod kısmı
file_path = input("Lütfen analiz etmek istediğiniz metin dosyasının yolunu girin: ").strip()

try:
    user_texts = parse_user_texts(file_path)

    if not user_texts:
        print("Dosyada analiz edilecek User metni bulunamadı.")
    else:
        for i, user_text in enumerate(user_texts, start=1):
            print(f"\n=== User {i} Metni Analizi ===")
            # Burada kendi analiz fonksiyonunu kullan:
            result = hybrid_analysis(user_text, models_tokenizers)

            print(f"\n BERT Modeli - Nihai Karar: {result['bert_result']['final_decision']} (Güven: %{result['bert_result']['confidence']*100:.1f})")
            print(f"BERT Modeli - Tüm Sınıf Olasılıkları: {result['bert_result']['all_class_probs']}")

            print(f"\n RoBERTa Modeli - Nihai Karar: {result['roberta_result']['final_decision']} (Güven: %{result['roberta_result']['confidence']*100:.1f})")
            print(f"RoBERTa Modeli - Tüm Sınıf Olasılıkları: {result['roberta_result']['all_class_probs']}")

            print(f"\n Ensemble Modeli - Nihai Karar: {result['ensemble_result']['final_decision']} (Güven: %{result['ensemble_result']['confidence']*100:.1f})")
            print(f"Ensemble Modeli - Tüm Sınıf Olasılıkları: {result['ensemble_result']['all_class_probs']}")

            if result["ensemble_result"]["alerts"]:
                print("\n ⚠️ Kritik Uyarılar:")
                for cls, prob in result["ensemble_result"]["alerts"]:
                    print(f"- {cls}: %{prob*100:.1f}")

except FileNotFoundError:
    print("Dosya bulunamadı. Lütfen geçerli bir yol girin.")


Lütfen analiz etmek istediğiniz metin dosyasının yolunu girin: /content/Ellie.txt

=== User 1 Metni Analizi ===

 BERT Modeli - Nihai Karar: bipolar (Güven: %78.6)
BERT Modeli - Tüm Sınıf Olasılıkları: {'depresyon': 0.006, 'intihar eğilimi': 0.006, 'anksiyete': 0.136, 'stres': 0.003, 'kişilik bozukluğu': 0.015, 'bipolar': 0.786, 'öfke': 0.001, 'hakaret': 0.031, 'tehdit': 0.0, 'nefret söylemi': 0.009, 'üzgünlük': 0.001, 'umutlu': 0.0, 'sevinç': 0.006, 'normal': 0.001}

 RoBERTa Modeli - Nihai Karar: anksiyete (Güven: %73.7)
RoBERTa Modeli - Tüm Sınıf Olasılıkları: {'depresyon': 0.031, 'intihar eğilimi': 0.091, 'anksiyete': 0.736, 'stres': 0.0, 'kişilik bozukluğu': 0.035, 'bipolar': 0.078, 'öfke': 0.0, 'hakaret': 0.002, 'tehdit': 0.0, 'nefret söylemi': 0.018, 'üzgünlük': 0.0, 'umutlu': 0.001, 'sevinç': 0.003, 'normal': 0.002}

 Ensemble Modeli - Nihai Karar: Emin Değil (Güven: %43.6)
Ensemble Modeli - Tüm Sınıf Olasılıkları: {'depresyon': 0.019, 'intihar eğilimi': 0.049, 'anksiyete': 0.4

In [ ]:
import json

# öneri6.json dosyasını okuyup
with open("öneri6.json", encoding="utf-8") as f:
    data = json.load(f)

# sadece tekil_duygular altındaki sözlüğü kullan
oneri_dict = data["tekil_duygular"]


def get_recommendations(prob_dict, threshold=0.10):
    """
    prob_dict: {class_label: probability}
    threshold: öneri vermek için olasılık alt sınırı
    """
    # 0.10 ve üzeri olan sınıfları al, olasılığa göre azalan sırala
    filtered = [(cls, prob) for cls, prob in prob_dict.items() if prob >= threshold]
    filtered.sort(key=lambda x: x[1], reverse=True)

    recommendations = []
    for cls, prob in filtered:
        # Anahtarları normalize ederek eşleşmeyi artır
        oneri = oneri_dict.get(cls.strip().lower(), None)
        if oneri:
            recommendations.append(f"{cls} (%{prob*100:.1f}): {oneri}")

    if not recommendations:
        recommendations.append("Yüksek olasılıklı duygu bulunamadı ya da öneri mevcut değil.")

    return recommendations

# Ana analiz döngüsündeki ensemble kısmına öneri ekleyelim:

for i, user_text in enumerate(user_texts, start=1):
    print(f"\n=== User {i} Metni Analizi ===")
    result = hybrid_analysis(user_text, models_tokenizers)

    print(f"\n BERT Modeli - Nihai Karar: {result['bert_result']['final_decision']} (Güven: %{result['bert_result']['confidence']*100:.1f})")
    print(f"BERT Modeli - Tüm Sınıf Olasılıkları: {result['bert_result']['all_class_probs']}")

    print(f"\n RoBERTa Modeli - Nihai Karar: {result['roberta_result']['final_decision']} (Güven: %{result['roberta_result']['confidence']*100:.1f})")
    print(f"RoBERTa Modeli - Tüm Sınıf Olasılıkları: {result['roberta_result']['all_class_probs']}")

    print(f"\n Ensemble Modeli - Nihai Karar: {result['ensemble_result']['final_decision']} (Güven: %{result['ensemble_result']['confidence']*100:.1f})")
    print(f"Ensemble Modeli - Tüm Sınıf Olasılıkları: {result['ensemble_result']['all_class_probs']}")

    # Kritik uyarılar varsa yazdır
    if result["ensemble_result"]["alerts"]:
        print("\n ⚠️ Kritik Uyarılar:")
        for cls, prob in result["ensemble_result"]["alerts"]:
            print(f"- {cls}: %{prob*100:.1f}")

    # Önerileri göster
    print("\n 📌 Öneriler:")
    recommendations = get_recommendations(result['roberta_result']['all_class_probs'], threshold=0.10)
    for rec in recommendations:
        print(f"- {rec}")



=== User 1 Metni Analizi ===

 BERT Modeli - Nihai Karar: bipolar (Güven: %78.6)
BERT Modeli - Tüm Sınıf Olasılıkları: {'depresyon': 0.006, 'intihar eğilimi': 0.006, 'anksiyete': 0.136, 'stres': 0.003, 'kişilik bozukluğu': 0.015, 'bipolar': 0.786, 'öfke': 0.001, 'hakaret': 0.031, 'tehdit': 0.0, 'nefret söylemi': 0.009, 'üzgünlük': 0.001, 'umutlu': 0.0, 'sevinç': 0.006, 'normal': 0.001}

 RoBERTa Modeli - Nihai Karar: anksiyete (Güven: %73.7)
RoBERTa Modeli - Tüm Sınıf Olasılıkları: {'depresyon': 0.031, 'intihar eğilimi': 0.091, 'anksiyete': 0.736, 'stres': 0.0, 'kişilik bozukluğu': 0.035, 'bipolar': 0.078, 'öfke': 0.0, 'hakaret': 0.002, 'tehdit': 0.0, 'nefret söylemi': 0.018, 'üzgünlük': 0.0, 'umutlu': 0.001, 'sevinç': 0.003, 'normal': 0.002}

 Ensemble Modeli - Nihai Karar: Emin Değil (Güven: %43.6)
Ensemble Modeli - Tüm Sınıf Olasılıkları: {'depresyon': 0.019, 'intihar eğilimi': 0.049, 'anksiyete': 0.436, 'stres': 0.002, 'kişilik bozukluğu': 0.025, 'bipolar': 0.432, 'öfke': 0.0, 'ha

In [ ]:
prob_dict = result['roberta_result']['all_class_probs']

In [ ]:
filtered = [(cls, prob) for cls, prob in prob_dict.items()]
filtered.sort(key=lambda x: x[1], reverse=True)


In [ ]:
for cls in result['roberta_result']['all_class_probs'].keys():
    print(f"Model sınıfı: '{cls}'")


Model sınıfı: 'depresyon'
Model sınıfı: 'intihar eğilimi'
Model sınıfı: 'anksiyete'
Model sınıfı: 'stres'
Model sınıfı: 'kişilik bozukluğu'
Model sınıfı: 'bipolar'
Model sınıfı: 'öfke'
Model sınıfı: 'hakaret'
Model sınıfı: 'tehdit'
Model sınıfı: 'nefret söylemi'
Model sınıfı: 'üzgünlük'
Model sınıfı: 'umutlu'
Model sınıfı: 'sevinç'
Model sınıfı: 'normal'


In [ ]:
filtered

[('depresyon', 0.43),
 ('normal', 0.38),
 ('umutlu', 0.076),
 ('sevinç', 0.071),
 ('tehdit', 0.029),
 ('hakaret', 0.006),
 ('kişilik bozukluğu', 0.003),
 ('anksiyete', 0.002),
 ('intihar eğilimi', 0.001),
 ('bipolar', 0.001),
 ('stres', 0.0),
 ('öfke', 0.0),
 ('nefret söylemi', 0.0),
 ('üzgünlük', 0.0)]

In [ ]:
result['bert_result']['all_class_probs']

{'depresyon': 0.006,
 'intihar eğilimi': 0.006,
 'anksiyete': 0.136,
 'stres': 0.003,
 'kişilik bozukluğu': 0.015,
 'bipolar': 0.786,
 'öfke': 0.001,
 'hakaret': 0.031,
 'tehdit': 0.0,
 'nefret söylemi': 0.009,
 'üzgünlük': 0.001,
 'umutlu': 0.0,
 'sevinç': 0.006,
 'normal': 0.001}